In [1]:
import os
from pathlib import Path
from glob import glob
import pandas as pd

In [2]:
current_path = Path(os.getcwd())
root = current_path.parent.absolute()
data_folder = str(root) + "\\raw data\\"

In [5]:
types = ["xlsx", "csv", "xls"]

os.chdir(data_folder)

tanguy = {}

print("-- XLSX FILES --")
for file_path in glob("*.xlsx"):
    file_name = file_path.split(".")[0]
    
    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        tanguy[sheet] = file.parse(sheet)
    print()
    
    
print("\n-- XLS FILES --") 
for file_path in glob("*.xls"):
    file_name = file_path.split(".")[0]
    
    print(file_name + " sheets:")
    
    file = pd.ExcelFile(file_path)
    for sheet in file.sheet_names:
        print("-->"+sheet)
        tanguy[sheet] = file.parse(sheet)
    print()
    
print("\n-- CSV FILES --") 
for file_path in glob("*.csv"):
    file_name = file_path.split(".")[0]
    print(file_name)
    file_df = pd.read_csv(file_path)
    tanguy[file_name] = file_df



-- XLSX FILES --
Betting Against Beta Original Paper Data sheets:
-->BAB factors
-->Disclosures

Quality Minus Junk Factors Daily sheets:
-->QMJ Factors
-->HML Devil


-- XLS FILES --
sentiment sheets:
-->SENTIMENT
-->CHART


-- CSV FILES --
F-F_Momentum_Factor_daily
F-F_Research_Data_5_Factors_2x3_daily
F-F_ST_Reversal_Factor_daily
